In [5]:
import random
import json
import os
from core.document_retrieval import VectorDB, Retriever
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
source = 'ars_electronica_prizewinners_ru.json'
path = 'not_posted.txt'

vector_db = VectorDB(source)
vector_db.load()
vector_db.create_db()
retriever = Retriever(vector_db)

with open(source, 'r', encoding='utf-8') as f:
    data = json.load(f)

with open(path, 'r') as f:
    not_posted = f.readline().split(',')

key = random.choice(not_posted)
print(f'Key is {key}')
artwork = data[key]

print("Source:\n")
print(artwork["description_ru"], "\n")

res = retriever.get_top_k(artwork['description_ru'], key, 3)

print("Recs:\n")
for example in res:
    print(example.page_content, "\n")

Key is 23722
Source:

С 1975 года я создаю движущиеся изображения с помощью новых технологий, особенно систем трехмерной компьютерной анимации. Моя работа сосредоточена на изучении естественных движений и роли человеческих жестов и выражений в языке общения. Я рассматриваю развитие новых технологий как расширение человеческого разума и тела и комментирую эту взаимосвязь. «Установившееся состояние» определяется как стабильное состояние, которое не меняется с течением времени или при котором изменение в одном направлении постоянно уравновешивается изменить в другом. Тонкие паттерны синхронных движений тела возникают, когда люди взаимодействуют друг с другом. Эти телесные ритмы, называемые ритмами взаимодействия, необходимы в каждом человеческом взаимодействии. Когда люди двигают своим телом и вытягивают конечности, они вычерчивают невидимые формы, окружающие их тела, определяя пространство, которое они занимают. Когда два человека взаимодействуют, эти пространственные паттерны также пере

In [25]:
res[0].metadata['img_list'][0]

'https://archive.aec.at/media/assets/827beff79946a4c6e04bc54b6e825425.jpg'

In [27]:
res[0].metadata['description']

"Nowadays it's common to use the word 'intelligent' instead of this word 'able'. This is not justifiable, as is mostly the case in the wording Artificial Intelligence. I want my art-objects to react and act in non-trivial ways. Although we use our intelligence when acting in a non-trivial way, most of the existing objects and programs named 'intelligent' have abilities which are still far removed from the complex we call intelligence. Intelligence, I think, includes the notion of the ability to react - in principle in an unlimited and meaningful way, to new events. Qualities like this are yet to be constructed although neural nets bear a great promise. That's why I don't want to use the word 'intelligent' for now. Not only out of respect for its actual meaning but especially because it manipulates us too much in directions not necessary to make art, namely in trying to make really intelligent objects. Intelligence, my ass, art is about 'interestingness'. That's why: 'able'.At this mome

In [28]:
art_name = res[0].metadata['name']
authors = res[0].metadata['authors']
year = res[0].metadata['year']
description = res[0].metadata['description']

def delete_apostrophe(text):
    return text.replace("'", "")

prompt = f'Here is a description of the artwork "{art_name}" by {authors} ' \
         f'created in {year} year. Please, provide 3 examples ' \
         f'of contemporary artworks and artists that are somehow similar with this artwork. ' \
         f'Explain your decision. Do not divide description and reasoning parts, ' \
         f'combine them into the one paragraph. Give your answer in Russian, ' \
         f'but do not translate the names of artists and artworks.' \
         f'Use Markdown to structure your answer. Make the names of artists and artworks bold.' \
         f'\nDescription:\n{delete_apostrophe(description)}' \
         f'also provide a short description of the picture attached to the article'

response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url":{
                       "url": res[0].metadata['img_list'][0]
                    },
                },
            ],
        }
    ],
    max_tokens=1000,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Произведение искусства "Adelbrecht" Мартина Спаньярда олицетворяет собой машину, переходящую в категорию "он", что отражает искусственный интеллект и взаимодействие с человеком. Сходные темы исследуют и другие современные художники. **Ken Rinaldo** с его проектом **"Autopoiesis"**, который включает в себя кинетические скульптуры, взаимодействующие с окружающими людьми и их движениями, демонстрирует как технологии могут "жить" и реагировать на человеческое присутствие. **Petra Cortright** со своим использованием цифровых медиа и интерактивных веб-инсталляций, таких как **"VVEBCAM"**, позволяет зрителям взаимодействовать с артом онлайн, что расширяет понятие присутствия и реакции произведения на внешние стимулы. **Rafael Lozano-Hemmer** и его **"Pulse Room"**, где световые инсталляции отзываются на пульс посетителей, также подчеркивают тему взаимодействия между человеком и объектом, совмещая техно